In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("Downloads/social_media_data.csv")

In [5]:
df.describe()

,caption_length,hashtag_count,posting_hour,likes,comments,shares
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,117.500000,6.100000,18.150000,226.250000,26.800000,10.650000
std,47.889676,2.954034,2.641272,125.811837,18.724035,11.753051
min,30.000000,1.000000,11.000000,60.000000,4.000000,1.000000
25%,87.500000,4.000000,17.750000,127.500000,10.000000,2.750000
50%,115.000000,6.000000,19.000000,205.000000,23.000000,6.000000
75%,152.500000,8.000000,20.000000,302.500000,37.750000,12.750000
max,200.000000,12.000000,21.000000,500.000000,70.000000,40.000000


In [9]:
#cari angka engangement
df["engagement"] = df["likes"] + df["comments"] + df["shares"]
encoder = LabelEncoder()
df["content_type_encoded"] = encoder.fit_transform(df["content_type"])

#klaster jenis konten
kmeans = KMeans(n_clusters=3, random_state=42)
df["content_cluster"] = kmeans.fit_predict(df[["caption_length", "hashtag_count", "engagement"]])

#model prediksi
X = df[["content_type_encoded", "caption_length", "hashtag_count", "posting_hour"]]
y = df["engagement"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)


C:\Users\Asus\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


RandomForestRegressor(n_estimators=200, random_state=42)

In [10]:
#fungsi rekomendasi 
def recommend_content(content_type, caption_length, hashtag_count):

    # Konversi content type ke angka
    ct_encoded = encoder.transform([content_type])[0]

    # Cek jam posting terbaik (berdasarkan rata-rata ER tiap jam)
    best_hour = df.groupby("posting_hour")["engagement"].mean().idxmax()

    # Prediksi engagement untuk jam terbaik
    input_data = pd.DataFrame({
        "content_type_encoded": [ct_encoded],
        "caption_length": [caption_length],
        "hashtag_count": [hashtag_count],
        "posting_hour": [best_hour]
    })

    predicted_er = model.predict(input_data)[0]

    # Cari cluster konten terbaik
    best_cluster = df.groupby("content_cluster")["engagement"].mean().idxmax()
    cluster_mean = df[df["content_cluster"] == best_cluster]["content_type"].mode()[0]

    return {
        "recommended_posting_hour": int(best_hour),
        "predicted_engagement": int(predicted_er),
        "best_content_cluster_type": str(cluster_mean),
        "note": "Posting at this hour usually performs best based on historical data."
    }
